In [1]:
import nltk
import sklearn
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords

In [ ]:
# nltk.download_shell()

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:
categories = fetch_20newsgroups().target_names

In [4]:
train= fetch_20newsgroups(subset="train" ,categories = categories) 

In [5]:
test= fetch_20newsgroups(subset="test" ,categories = categories) 

In [6]:
df = pd.DataFrame(train["data"], columns=["text"])
df["target"] = train["target"]
print(df.head())

df_test = pd.DataFrame(test["data"], columns=["text"])
df_test["target"] = test["target"]
print(df_test.head())

                                                text  target
0  From: lerxst@wam.umd.edu (where's my thing)\nS...       7
1  From: guykuo@carson.u.washington.edu (Guy Kuo)...       4
2  From: twillis@ec.ecn.purdue.edu (Thomas E Will...       4
3  From: jgreen@amber (Joe Green)\nSubject: Re: W...       1
4  From: jcm@head-cfa.harvard.edu (Jonathan McDow...      14
                                                text  target
0  From: v064mb9k@ubvmsd.cc.buffalo.edu (NEIL B. ...       7
1  From: Rick Miller <rick@ee.uwm.edu>\nSubject: ...       5
2  From: mathew <mathew@mantis.co.uk>\nSubject: R...       0
3  From: bakken@cs.arizona.edu (Dave Bakken)\nSub...      17
4  From: livesey@solntze.wpd.sgi.com (Jon Livesey...      19


In [7]:
df["text"] = df["text"].str.lower()
df_test["text"] = df_test["text"].str.lower()

In [8]:
stop = stopwords.words('english')

In [9]:
url = re.compile(r'https?://\S+|www\.\S+')
df['text'] = df['text'].apply(lambda x : url.sub(r'', x))
df_test["text"] = df_test["text"].apply(lambda x : url.sub(r'', x))

In [10]:
html = re.compile(r'<.*?>')
df['text'] = df['text'].apply(lambda x: html.sub(r'', x))
df_test["text"] = df_test["text"].apply(lambda x: html.sub(r'', x))

In [11]:
table = str.maketrans('', '', string.punctuation)
df['text'] = df['text'].apply(lambda x: x.translate(table))
df_test["text"] = df_test["text"].apply(lambda x: x.translate(table))

In [12]:
df.head()

,text,target
0,from lerxstwamumdedu wheres my thing\nsubject ...,7
1,from guykuocarsonuwashingtonedu guy kuo\nsubje...,4
2,from twillisececnpurdueedu thomas e willis\nsu...,4
3,from jgreenamber joe green\nsubject re weitek ...,1
4,from jcmheadcfaharvardedu jonathan mcdowell\ns...,14


In [13]:
df_test.head()

,text,target
0,from v064mb9kubvmsdccbuffaloedu neil b gandler...,7
1,from rick miller \nsubject xface\norganization...,5
2,from mathew \nsubject re strong weak atheism\...,0
3,from bakkencsarizonaedu dave bakken\nsubject r...,17
4,from liveseysolntzewpdsgicom jon livesey\nsubj...,19


In [14]:
df["text"] = df["text"].apply(lambda line: line[5:])
df_test["text"] = df_test["text"].apply(lambda line: line[5:])

In [15]:
df["text"] = df["text"].apply(lambda line: line.replace("\n", " "))
df_test["text"] = df_test["text"].apply(lambda line: line.replace("\n", " "))

In [16]:
df["text"]

0        lerxstwamumdedu wheres my thing subject what c...
1        guykuocarsonuwashingtonedu guy kuo subject si ...
2        twillisececnpurdueedu thomas e willis subject ...
3        jgreenamber joe green subject re weitek p9000 ...
4        jcmheadcfaharvardedu jonathan mcdowell subject...
                               ...                        
11309    jimzisfeinfactorycom jim zisfein  subject re m...
11310    ebodinpearltuftsedu subject screen death mac p...
11311    westesnetcomcom will estes subject mounting cp...
11312    stevehcrlgw steven collins subject re sphere f...
11313    gunningccocaltechedu kevin j gunning subject s...
Name: text, Length: 11314, dtype: object

In [17]:
df_test["text"]

0       v064mb9kubvmsdccbuffaloedu neil b gandler subj...
1       rick miller  subject xface organization just m...
2       mathew  subject re strong  weak atheism organi...
3       bakkencsarizonaedu dave bakken subject re saud...
4       liveseysolntzewpdsgicom jon livesey subject re...
                              ...                        
7527    richmondspiffprincetonedu stupendous man subje...
7528    smytonjmurr11allegedu jim smyton subject re mo...
7529    hhendersonvaxclarkuedu subject re game length ...
7530    b859zamutarlgutaedu  subject intel chmos 80868...
7531    adamsjgtewdmtvgtegsccom subject re homosexuali...
Name: text, Length: 7532, dtype: object

In [18]:
def text_process(text):
    """
    1. Remove punctuations
    2. Remove stop words
    3. Return list of clean text words
    """
    
    return [word for word in text.split() if word.lower() not in stopwords.words('english')]

In [19]:
df["text"] = df["text"].apply(text_process)

In [20]:
df_test["text"] = df_test["text"].apply(text_process)

In [21]:
def list_to_string(text):
    line = ""
    for word in text:
        line += word + " "
    return line.strip()

In [22]:
df["text"] = df["text"].apply(list_to_string)

In [23]:
df_test["text"] = df_test["text"].apply(list_to_string)

In [24]:
df.head()

,text,target
0,lerxstwamumdedu wheres thing subject car nntpp...,7
1,guykuocarsonuwashingtonedu guy kuo subject si ...,4
2,twillisececnpurdueedu thomas e willis subject ...,4
3,jgreenamber joe green subject weitek p9000 org...,1
4,jcmheadcfaharvardedu jonathan mcdowell subject...,14


In [25]:
df_test.head()

,text,target
0,v064mb9kubvmsdccbuffaloedu neil b gandler subj...,7
1,rick miller subject xface organization lines 1...,5
2,mathew subject strong weak atheism organizatio...,0
3,bakkencsarizonaedu dave bakken subject saudi c...,17
4,liveseysolntzewpdsgicom jon livesey subject 20...,19


In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(df["text"])

In [28]:
text_bow = bow_transformer.transform(df["text"])

In [29]:
print(text_bow.shape)

(11314, 133383)


In [30]:
text_bow.nnz

1310714

In [31]:
test_bow = bow_transformer.transform(df_test["text"])

In [32]:
print(test_bow.shape)

(7532, 133383)


In [33]:
test_bow.nnz

795332

In [34]:
from sklearn.feature_extraction.text import TfidfTransformer

In [35]:
tfidf_transformer = TfidfTransformer().fit(text_bow)

In [36]:
text_tfidf = tfidf_transformer.transform(text_bow)

In [38]:
test_tfidf = tfidf_transformer.transform(test_bow)

In [81]:
from sklearn.naive_bayes import MultinomialNB

In [82]:
nb_model = MultinomialNB().fit(text_tfidf, df["target"])

In [83]:
all_pred_nb = nb_model.predict(test_tfidf)

In [84]:
from sklearn.neighbors import NearestCentroid

In [85]:
rocchio_model = NearestCentroid().fit(text_tfidf, df["target"])

In [86]:
all_pred_rc = rocchio_model.predict(test_tfidf)

In [87]:
from sklearn.neighbors import KNeighborsClassifier

In [88]:
knn_model = KNeighborsClassifier().fit(text_tfidf, df["target"])

In [89]:
all_pred_knn = knn_model.predict(test_tfidf)

In [90]:
from sklearn.metrics import f1_score

In [91]:
print("f1 score for Naive Bayse classifier")
f1_score(df_test["target"], all_pred_nb, average='micro')

f1 score for Naive Bayse classifier


0.8191715347849177

In [92]:
print("f1 score for Rocchio classifier")
f1_score(df_test["target"], all_pred_rc, average='micro')

f1 score for Rocchio classifier


0.7573021773765268

In [93]:
print("f1 score for KNN classifier")
f1_score(df_test["target"], all_pred_knn, average='micro')

f1 score for KNN classifier


0.7428305894848647